In [12]:
%load_ext autoreload
%autoreload 2

import torchvision.models as models
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T
import numpy as np
import pandas
import matplotlib.pyplot as plt

import lightning as L
from lightning.pytorch.loggers import TensorBoardLogger

import torchmetrics
import webdataset as wds

import label_mapping

TORCH_ACCELERATOR = "cpu"

torch.cuda.get_device_name(0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'NVIDIA GeForce GTX 1070'

In [15]:
%run datasets.ipynb

DATASET_ROOT = Path.home() / "datasets" / "im2gps" / "outputs"

# Load the s2cell-annotated dataset
annotated_df = pandas.read_pickle(DATASET_ROOT / "s2cell_2007" / "annotated.pkl")
mapping = label_mapping.LabelMapping.read_csv(DATASET_ROOT / "s2cell_2007" / "cells.csv")

mapping

In [20]:

image_id_to_s2cell = {row.id: row.s2cell for row in annotated_df.itertuples()}

# total set has ~630k images
BATCH_SIZE = auto_batch_size()
print("Batch size:", BATCH_SIZE)

dataset = wds.WebDataset(
        str(DATASET_ROOT / "wds" / "im2gps_2007_{000..038}.tar"),
        shardshuffle=True,
    ).decode("torchrgb").to_tuple("jpg", "json")

# Transform s2cells to labels, skipping examples without s2cell
def to_img_label(sample):
    img, meta = sample
    s2cell = image_id_to_s2cell.get(meta["id"])
    if s2cell is None:
        raise NotImplementedError("Skipping example without s2cell")
    label = mapping.get_label(s2cell)
    # TODO: this is where we transform the image
    return img, label

#dataset = dataset.map(to_img_label, handler=wds.ignore_and_continue)
dataset = dataset.map(to_img_label)
dataset = dataset.batched(BATCH_SIZE)

# Visualize a few loaded samples
dataloader = wds.WebLoader(dataset, batch_size=None, num_workers=0)
for inputs, targets in dataloader:
    print(inputs.shape, targets.shape, targets)
    break

Unknown device: NVIDIA GeForce GTX 1070
Batch size: 1
torch.Size([1, 3, 477, 1024]) torch.Size([1]) tensor([1237])


In [ ]:

# Define a LightningModule for the classifier
class NcalScalClassifierMnet3(L.LightningModule):
    def __init__(self):
        super().__init__()

        mnet3 = models.mobilenet_v3_large(weights="IMAGENET1K_V2")

        self.features = mnet3.features
        self.avgpool = mnet3.avgpool
        self.classifier = nn.Sequential(
            nn.Linear(mnet3.classifier[0].in_features, 1280),
            nn.Hardswish(inplace=True),
            nn.Dropout(p=0.2, inplace=True),
            nn.Linear(1280, 2),
        )

        torch.nn.init.xavier_uniform_(self.classifier[0].weight)
        torch.nn.init.xavier_uniform_(self.classifier[3].weight)

        self.accuracy = torchmetrics.classification.Accuracy(task='multiclass', num_classes=2)

    def forward(self, x):
        with torch.no_grad():
            x = self.features(x)
            x = self.avgpool(x)
            x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        z = self.forward(x)
        self.accuracy(z, y)
        self.log('train_acc_step', self.accuracy, prog_bar=True)

        loss = nn.CrossEntropyLoss()(z, y)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        z = self.forward(x)
        self.accuracy(z, y)
        self.log('test_acc', self.accuracy, on_step=False, on_epoch=True)

        test_loss = nn.CrossEntropyLoss()(z, y)
        self.log("test_loss", test_loss, prog_bar=True)
        return test_loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.001)